<a href="https://colab.research.google.com/github/hahahonline/YOLOv4-Deteccao-de-objetos-Videos-Darknet/blob/main/(meu)_YOLO_v4_Detec%C3%A7%C3%A3o_de_objetos_V%C3%ADdeos_Darknet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Detectando objetos em videos com YOLOv4

## Etapa 1 - Download do Darknet

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15514, done.
remote: Total 15514 (delta 0), reused 0 (delta 0), pack-reused 15514
Receiving objects: 100% (15514/15514), 14.17 MiB | 15.90 MiB/s, done.
Resolving deltas: 100% (10412/10412), done.


In [ ]:
%cd darknet

/content/darknet


In [ ]:
ls

3rdparty/               darknet_video.py        net_cam_v4.sh*
build/                  data/                   README.md
build.ps1*              image_yolov3.sh*        results/
cfg/                    image_yolov4.sh*        scripts/
cmake/                  include/                src/
CMakeLists.txt          json_mjpeg_streams.sh*  vcpkg.json
DarknetConfig.cmake.in  LICENSE                 vcpkg.json.opencv23
darknet_images.py       Makefile                video_yolov3.sh*
darknet.py              net_cam_v3.sh*          video_yolov4.sh*


## Etapa 2 - Compilando a biblioteca

In [ ]:
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
print(device_name)

/device:GPU:0


In [ ]:
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
  946 |                 float rgb[3];
      |                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
 1147 |             if (iteration_old == 0)
      |

## Etapa 3 - Carregando os pesos do modelo pré-treinado

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

In [ ]:
#este código está descompactando um arquivo zip localizado no caminho especificado 
#e extraindo todo o seu conteúdo para o diretório atual.
import zipfile
path = "/content/gdrive/My Drive/Cursos/YOLO/modelo_YOLOv4.zip"
zip_object = zipfile.ZipFile(file=path, mode="r")
zip_object.extractall("./")
zip_object.close()

## Etapa 4 - Carregando o vídeo

### 4.1 - De uma url

In [ ]:
!wget https://github.com/gabevr/yolo/raw/master/videos/video_teste02.mp4

--2020-10-07 17:29:36--  https://github.com/gabevr/yolo/raw/master/videos/video_teste02.mp4
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/gabevr/yolo/master/videos/video_teste02.mp4 [following]
--2020-10-07 17:29:37--  https://raw.githubusercontent.com/gabevr/yolo/master/videos/video_teste02.mp4
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7266878 (6.9M) [application/octet-stream]
Saving to: ‘video_teste02.mp4’

video_teste02.mp4   100%[===================>]   6.93M  35.2MB/s    in 0.2s    

2020-10-07 17:29:38 (35.2 MB/s) - ‘video_teste02.mp4’ saved [7266878/7266878]



### 4.2 - Do Google Drive


In [ ]:
#Esse comando usa a ferramenta cp do sistema operacional Unix para copiar um arquivo de um diretório do Google Drive para o diretório atual (./), que é o diretório raiz do ambiente de execução atual no Colab.
#O ./ é um caminho relativo que indica o diretório atual em que o comando está sendo executado.
!cp /content/gdrive/My\ Drive/Cursos/YOLO/videos/uniceub.mp4 ./

In [ ]:
ls

3rdparty/               darknet_video.py        README.md
backup/                 data/                   results/
build/                  image_yolov3.sh*        scripts/
build.ps1*              image_yolov4.sh*        src/
cfg/                    include/                vcpkg.json
cmake/                  json_mjpeg_streams.sh*  vcpkg.json.opencv23
CMakeLists.txt          LICENSE                 video_yolov3.sh*
darknet*                Makefile                video_yolov4.sh*
DarknetConfig.cmake.in  net_cam_v3.sh*          yolov4.weights
darknet_images.py       net_cam_v4.sh*
darknet.py              obj/


## Etapa 5 - Realizando a detecção em vídeo

In [ ]:
#detector demo = executar o modelo YOLO em um arquivo de vídeo ou webcam ao vivo para detectar objetos em tempo real
#yolov4.cfg = estrutura da rede neural
#yolov4.weights = pesos 
#dont_show = pra nao mostrar o processamento
#indica que o Darknet deve usar o dispositivo de entrada de vídeo número 0, que é geralmente a webcam padrão do computador. (não vai funcionar pq eu passei um video na entrada)
!./darknet detector demo cfg/coco.data cfg/yolov4.cfg yolov4.weights -dont_show uniceub.mp4 -i 0 -out_filename resultado_ceub.avi

A saída de streaming foi truncada nas últimas 5000 linhas.
traffic light: 38% 
traffic light: 27% 
truck: 53% 
car: 61% 
truck: 31% 
car: 100% 
car: 100% 
car: 96% 
car: 94% 
car: 93% 
car: 93% 
car: 92% 
car: 85% 
car: 44% 

FPS:19.6 	 AVG_FPS:20.0

 cvWriteFrame 
Objects:

traffic light: 97% 
traffic light: 92% 
traffic light: 56% 
traffic light: 32% 
truck: 61% 
car: 53% 
truck: 31% 
car: 88% 
car: 100% 
car: 100% 
car: 97% 
car: 97% 
car: 94% 
car: 94% 
car: 86% 
car: 50% 
car: 28% 

FPS:19.6 	 AVG_FPS:20.0

 cvWriteFrame 
Objects:

traffic light: 97% 
traffic light: 93% 
traffic light: 64% 
traffic light: 44% 
truck: 58% 
car: 52% 
car: 100% 
car: 100% 
car: 98% 
car: 97% 
car: 97% 
car: 96% 
car: 95% 
car: 91% 
car: 39% 

FPS:19.6 	 AVG_FPS:20.0

 cvWriteFrame 
Objects:

traffic light: 97% 
traffic light: 93% 
traffic light: 64% 
traffic light: 44% 
truck: 58% 
car: 51% 
car: 100% 
car: 100% 
car: 98% 
car: 97% 
car: 97% 
car: 96% 
car: 95% 
car: 91% 
car: 39% 

FPS:19.5 	 AVG_FP

In [ ]:
#mostrando o tamanho do arquivo
!du -h resultado.avi

32M	resultado.avi


## Etapa 6 - Visualizando o resultado

In [ ]:
!cp ./resultado.avi /content/gdrive/My\ Drive/Cursos/YOLO/videos/resultado1.avi

## Especificando um threshold

In [ ]:
!wget https://github.com/gabevr/yolo/raw/master/videos/video_rua01.mp4

--2020-10-07 17:51:00--  https://github.com/gabevr/yolo/raw/master/videos/video_rua01.mp4
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/gabevr/yolo/master/videos/video_rua01.mp4 [following]
--2020-10-07 17:51:01--  https://raw.githubusercontent.com/gabevr/yolo/master/videos/video_rua01.mp4
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5645465 (5.4M) [application/octet-stream]
Saving to: ‘video_rua01.mp4’

video_rua01.mp4     100%[===================>]   5.38M  23.8MB/s    in 0.2s    

2020-10-07 17:51:01 (23.8 MB/s) - ‘video_rua01.mp4’ saved [5645465/5645465]



In [ ]:
#faz uma copia do video do meu diretorio do drive  
!cp /content/gdrive/My\ Drive/Cursos/YOLO/videos/resultado_uniceub.avi ./

In [ ]:
#detector demo = executar o modelo YOLO em um arquivo de vídeo ou webcam ao vivo para detectar objetos em tempo real
#yolov4.cfg = estrutura da rede neural
#yolov4.weights = pesos 
#dont_show = pra nao mostrar o processamento
#thresh = nivel de confiança da detecção de objeto 
!./darknet detector demo cfg/coco.data cfg/yolov4.cfg yolov4.weights -dont_show resultado_uniceub.avi -out_filename resultado_uniceub2.avi -thresh 0.2

A saída de streaming foi truncada nas últimas 5000 linhas.
person: 22% 

FPS:19.8 	 AVG_FPS:20.8

 cvWriteFrame 
Objects:


FPS:19.8 	 AVG_FPS:20.8

 cvWriteFrame 
Objects:


FPS:19.9 	 AVG_FPS:20.8

 cvWriteFrame 
Objects:


FPS:19.9 	 AVG_FPS:20.8

 cvWriteFrame 
Objects:

person: 29% 
person: 24% 

FPS:20.0 	 AVG_FPS:20.8

 cvWriteFrame 
Objects:

person: 31% 
person: 28% 
person: 25% 
person: 24% 
person: 24% 
person: 21% 

FPS:20.1 	 AVG_FPS:20.8

 cvWriteFrame 
Objects:

person: 43% 
person: 37% 
person: 32% 

FPS:19.8 	 AVG_FPS:20.8

 cvWriteFrame 
Objects:

person: 23% 
person: 21% 
person: 21% 
person: 20% 

FPS:20.0 	 AVG_FPS:20.8

 cvWriteFrame 
Objects:


FPS:19.9 	 AVG_FPS:19.7

 cvWriteFrame 
Objects:


FPS:20.0 	 AVG_FPS:19.7

 cvWriteFrame 
Objects:

person: 23% 

FPS:20.0 	 AVG_FPS:19.7

 cvWriteFrame 
Objects:


FPS:20.0 	 AVG_FPS:19.7

 cvWriteFrame 
Objects:


FPS:19.8 	 AVG_FPS:19.7

 cvWriteFrame 
Objects:


FPS:20.0 	 AVG_FPS:19.7

 cvWriteFrame 
Objects:

person

In [ ]:
#copiando o video na pasta do google drive
!cp ./resultado_uniceub2.avi /content/gdrive/My\ Drive/Cursos/YOLO/videos/resultado_uniceub2.avi